Generate synthetic data that looks like my input data.
The input has fields:
Ticker
Investor name
units
value

In [2]:
import pandas as pd
import random

In [10]:
def generate_synthetic_data(tickers_list, investor_name_list, calendar_date_list, n):
    data = set()

    while len(data) < n:
        ticker = random.choice(tickers_list)
        investor_name = random.choice(investor_name_list)
        calendar_date = random.choice(calendar_date_list)
        own = 1

        entry = (ticker, investor_name, calendar_date, own)
        data.add(entry)

    df = pd.DataFrame(data, columns=['ticker', 'investorname', 'calendardate', 'own'])
    return df

# Example usage
tickers_list = ['AAPL', 'GOOG', 'TSLA', 'AMZN']
investor_name_list = ['Investor_A', 'Investor_B', 'Investor_C', 'Investor_D']
calendar_date_list = ['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01']
n = 10

df = generate_synthetic_data(tickers_list, investor_name_list, calendar_date_list, n)
print(df)

  ticker investorname calendardate  own
0   GOOG   Investor_B   2021-02-01    1
1   GOOG   Investor_A   2021-04-01    1
2   AMZN   Investor_C   2021-01-01    1
3   GOOG   Investor_A   2021-02-01    1
4   TSLA   Investor_B   2021-02-01    1
5   AMZN   Investor_B   2021-03-01    1
6   AAPL   Investor_B   2021-04-01    1
7   AAPL   Investor_D   2021-02-01    1
8   AMZN   Investor_B   2021-02-01    1
9   TSLA   Investor_D   2021-02-01    1


read the investor detail file
we can take the top n investors, tickers and all of the dates and then generate synthetic data


In [6]:
#1read the sf3 file data
#2 pick 10 unique investor names
#3 pick 10 unique tickers
#4 pick 10 un ique calendar dates
#5 generate 1000 random entries


# Path: stock_colab_filter/matrix_factorize_v2.ipynb
import pandas as pd
import random

#1 read the sf3 file data
# Read SF3 table from CSV file
sf3 = pd.read_csv('data/SHARADAR_holdings.csv')
sf3.head()


,ticker,investorname,securitytype,calendardate,value,units,price
0,EAR,PATIENT SQUARE CAPITAL LP,SHR,2022-12-31,1.819449e+11,3.164260e+08,575.0
1,AAPL,VANGUARD GROUP INC,SHR,2022-12-31,1.660831e+11,1.278251e+09,129.0
2,MSFT,VANGUARD GROUP INC,SHR,2022-12-31,1.541239e+11,6.426649e+08,239.0
3,AAPL,BLACKROCK INC,SHR,2022-12-31,1.337250e+11,1.029208e+09,129.0
4,MSFT,BLACKROCK INC,SHR,2022-12-31,1.275945e+11,5.320427e+08,239.0


In [23]:
#2 pick 10 unique investor names
investor_name_list = sf3['investorname'].unique()
investor_name_list = investor_name_list[:10]
investor_name_list
#3 pick 10 unique tickers
tickers_list = sf3['ticker'].unique()
tickers_list = tickers_list[:10]
tickers_list
#4 pick 10 unique calendar dates
calendar_date_list = sf3['calendardate'].unique()
calendar_date_list = calendar_date_list[:10]
calendar_date_list
#5 generate 1000 random entries
n = 100
df = generate_synthetic_data(tickers_list, investor_name_list, calendar_date_list, n)
print(df)


   ticker                 investorname calendardate  own
0     XOM            STATE STREET CORP   2021-03-31    1
1     XOM       BERKSHIRE HATHAWAY INC   2021-06-30    1
2    AAPL           VANGUARD GROUP INC   2020-12-31    1
3    MSFT         SABBY MANAGEMENT LLC   2021-06-30    1
4   GOOGL           VANGUARD GROUP INC   2022-03-31    1
..    ...                          ...          ...  ...
95  GOOGL            STATE STREET CORP   2021-03-31    1
96    XOM  PRICE T ROWE ASSOCIATES INC   2021-12-31    1
97   AAPL                      FMR LLC   2022-12-31    1
98   MSFT             BANK OF MONTREAL   2021-12-31    1
99   AMZN       BERKSHIRE HATHAWAY INC   2022-06-30    1

[100 rows x 4 columns]


In [34]:
import pandas as pd

def insert_correlated_stocks(df, correlated_pairs):
    # Copy the original DataFrame to avoid modifying it
    new_df = df.copy()

    # Iterate through the correlated_pairs
    for stock1, stock2 in correlated_pairs:
        # Find rows with stock1 or stock2 in the 'ticker' column
        stock1_rows = new_df[new_df['ticker'] == stock1]
        stock2_rows = new_df[new_df['ticker'] == stock2]

        # Create new rows for the correlated stocks
        new_stock1_rows = stock2_rows.copy()
        new_stock2_rows = stock1_rows.copy()

        # Update the 'ticker' column with the correlated stock
        new_stock1_rows['ticker'] = stock1
        new_stock2_rows['ticker'] = stock2

        # Append the new rows to the DataFrame
        new_df = pd.concat([new_df, new_stock1_rows, new_stock2_rows], ignore_index=True)

    # Remove any potential duplicates
    new_df = new_df.drop_duplicates(subset=['ticker', 'investorname', 'calendardate'])

    return new_df
correlated_pairs = [(tickers_list[0],tickers_list[1]), (tickers_list[2], tickers_list[3])]

df=insert_correlated_stocks(df, correlated_pairs)

compute some kind of manual correlation metric

In [33]:
import pandas as pd

def compute_correlation(df_group):
    ownership_matrix = df_group.pivot_table(index='investorname', columns='ticker', values='own', fill_value=0)
    correlation_matrix = ownership_matrix.corr(method='pearson')
    return correlation_matrix

# Group the dataset by calendardate
grouped_data = df.groupby('calendardate')

# Compute the pairwise correlation between investors for each calendar date
correlation_matrices = grouped_data.apply(compute_correlation)

# Print correlation matrices for each calendar date
for date, matrix in correlation_matrices.groupby(level=0):
    print(f"Correlation matrix for date {date}")
    print(matrix)

Correlation matrix for date 2020-09-30
ticker                   AAPL      AMZN       EAR     GOOGL       JNJ   
calendardate ticker                                                     
2020-09-30   AAPL    1.000000 -0.292770  1.000000 -0.066667  0.487950  \
             AMZN   -0.292770  1.000000 -0.292770 -0.292770 -0.142857   
             EAR     1.000000 -0.292770  1.000000 -0.066667  0.487950   
             GOOGL  -0.066667 -0.292770 -0.066667  1.000000 -0.292770   
             JNJ     0.487950 -0.142857  0.487950 -0.292770  1.000000   
             MSFT   -0.066667 -0.292770 -0.066667  1.000000 -0.292770   
             ROIV    0.149071 -0.218218  0.149071 -0.447214 -0.218218   
             UNH    -0.292770 -0.142857 -0.292770 -0.292770 -0.142857   
             XOM    -0.292770 -0.142857 -0.292770 -0.292770 -0.142857   

ticker                   MSFT      ROIV       UNH       XOM  BRK.B  
calendardate ticker                                                 
2020-09-30   AAPL  

In [29]:
#lets add a correlation on tickers
# if a ticker is owned by investor A, then it is also owned by investor B
correlated_pairs = [(tickers_list[0],tickers_list[1]), (tickers_list[2], tickers_list[3])]
#go through each row in the dataframe
#if the ticker is in the correlated pairs, then add a row with the same investor name and calendar date and the other tickeri in the touple
for index, row in df.iterrows():
    if (row['ticker'], row['investorname'], row['calendardate']) in correlated_pairs:
        #determine the other ticker
        

        #other_ticker = correlated_pairs[0][1] if correlated_pairs[0][0] == row['ticker'] else correlated_pairs[0][0]


        #df.loc[len(df)] = [row['ticker'], row['investorname'], row['calendardate'], 1]
